In [1]:
# EDA for filtering

In [10]:
import pandas as pd
games = pd.read_csv("C:/Users/toris/Downloads/Spring 25/DS4420/Final Project/game_data/games.csv") 

In [11]:
games.head(1)

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,...,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
0,1,Die Macher,die macher game seven sequential political rac...,1986,4.3206,7.61428,7.10363,1.57979,3,5,...,21926,21926,0,1,0,0,0,0,0,0


In [2]:
# imports
import pandas as pd
subcategories = pd.read_csv("C:/Users/toris/Downloads/Spring 25/DS4420/Final Project/game_data/subcategories.csv")
games = pd.read_csv("C:/Users/toris/Downloads/Spring 25/DS4420/Final Project/game_data/games.csv") 
ratings = pd.read_csv("C:/Users/toris/Downloads/Spring 25/DS4420/Final Project/game_data/user_ratings.csv") 

# initialize list
cats = ["All", "Thematic", "Strategy", "War", "Family", "CGS", "Abstract", "Party", "Childrens"]
sub_cats = ["All", "Exploration", "Miniatures", "Territory Building", "Card Game", "Educational", "Puzzle", "Collectible Components", "Word Game", "Print & Play", "Electronic"]

# rename columns
games.columns = [col.replace("Cat:", "").strip() if 40 <= i <= 48 else col for i, col in enumerate(games.columns)]

# filter cols
games = games.iloc[:, [0, 1] + list(range(40, 48))]

# create full data 
full_games = pd.merge(games, subcategories, on="BGGId", how="inner") 

# generate categories list and number of categories per game
full_games["Categories"] = full_games.iloc[:, 2:10].apply(lambda row: ', '.join(full_games.columns[2:10][row == 1]), axis=1)
full_games["Num_Categories"] = full_games.iloc[:, 2:10].sum(axis=1)

# generate sub categories list and number of categories per game
full_games["Sub Categories"] = full_games.iloc[:, 10:20].apply(lambda row: ', '.join(full_games.columns[10:20][row == 1]), axis=1)
full_games["Num_Sub_Categories"] = full_games.iloc[:, 10:20].sum(axis=1)

# Filter out rows where Num_Categories is 0
full_games = full_games[full_games["Num_Categories"] > 0]

# NEW CAT CODE
unique_categories = set(full_games["Categories"].str.split(", ").explode().dropna().replace("", float("nan")).dropna())
categories_list = sorted(unique_categories)

print("Categories:")
for i, category in enumerate(categories_list, 1):  
    print(f"{i}. {category}")

while True:
    try:
        categories = list(map(int, input("Select categories of interest by entering the corresponding number(s): ").split()))

        # Validate input
        if len(categories) >= 1 and all(1 <= choice <= len(categories_list) for choice in categories):
            break  
        else:
            print("\nInvalid selection. Please enter a valid number from the list.")
    except ValueError:
        print("\nInvalid input. Please enter numbers only.")

# Display selected games
category_names = []

print("You selected")
for i in categories:
    selected_category = categories_list[i - 1]
    category_names.append(selected_category)
    print(selected_category)

filtered_games = full_games[full_games[category_names].any(axis=1)]

# NEW SUB CAT CODE
y_n = input("Would you like to select a subcategory? (Y/N) ")

if y_n.lower()== "y":
    unique_sub_categories = set(filtered_games["Sub Categories"].str.split(", ").explode().dropna().replace("", float("nan")).dropna())
    sub_categories_list = sorted(unique_sub_categories)

    print("Sub-Categories:")
    for i, category in enumerate(sub_categories_list, 1):  
        print(f"{i}. {category}")

    while True:
        try:
            sub_categories = list(map(int, input("Select sub-categories of interest by entering the corresponding number(s): ").split()))
    
            # Validate input
            if len(sub_categories) >= 1 and all(1 <= choice <= len(sub_categories_list) for choice in sub_categories):
                break  
            else:
                print("\nInvalid selection. Please enter a valid number from the list.")
        except ValueError:
            print("\nInvalid input. Please enter numbers only.")
    
    # Display selected games
    sub_category_names = []
    
    print("You selected")
    for i in sub_categories:
        selected_sub_category = sub_categories_list[i - 1]
        sub_category_names.append(selected_sub_category)
        print(selected_sub_category)
    
    filtered_games = filtered_games[filtered_games[sub_category_names].any(axis=1)]

# Reset index to start from 1
filtered_games.insert(0, "Game Number", range(1, len(filtered_games) + 1))

# get favorite games from user
print("Game Choices:")
for _, row in filtered_games.iterrows():
    print(f"{row['Game Number']}. {row['Name']}")

while True:
    try:
        fav_numbers = list(map(int, input("Enter the Game Numbers of your favorite Games separated by spaces: ").split()))

        # Validate input
        if len(fav_numbers) >= 1 and all(num in filtered_games["Game Number"].values for num in fav_numbers):
            break  
        else:
            print("\nInvalid selection. Please enter a valid number from the list.")
    except ValueError:
        print("\nInvalid input. Please enter numbers only.")

# Display selected games
print("\nYour Favorite Game(s):")
for _, row in filtered_games[filtered_games["Game Number"].isin(fav_numbers)].iterrows():
    print(row['Name'])

while True:
    try:
        least_fav_numbers = list(map(int, input("Enter the Game Numbers of your least favorite Games separated by spaces: ").split()))

        # Validate input
        if all(num in filtered_games["Game Number"].values for num in least_fav_numbers):
            break  
        else:
            print("\nInvalid selection. Please enter exactly 3 valid numbers from the list.")
    except ValueError:
        print("\nInvalid input. Please enter numbers only.")

# Display selected games
print("\nYour Least Favorite Game(s):")
for _, row in filtered_games[filtered_games["Game Number"].isin(least_fav_numbers)].iterrows():
    print(row['Name'])

# get id lists for filtering ratings
favorite_game_id = filtered_games[filtered_games["Game Number"].isin(fav_numbers)]["BGGId"].tolist()
least_favorite_game_id = filtered_games[filtered_games["Game Number"].isin(least_fav_numbers)]["BGGId"].tolist()
filtered_game_ids = filtered_games["BGGId"]

#create ratings for the user
new_ratings = pd.DataFrame({
    "BGGId": favorite_game_id + least_favorite_game_id,
    "Rating": [5] * len(favorite_game_id) + [1] * len(least_favorite_game_id),
    "Username": ["TestUser"] * (len(favorite_game_id) + len(least_favorite_game_id))
})

# Append the new ratings to the existing DataFrame
ratings = pd.concat([ratings, new_ratings], ignore_index=True)

#get user input
y_n = input("Would you like recommendations for only the categories you have chosen? (Y/N) ")

# filter the df is needed
if y_n.lower()== "y":
    ratings = ratings[ratings["BGGId"].isin(filtered_game_ids)]

rating_matrix = ratings.pivot_table(index="Username", columns="BGGId", values="Rating", aggfunc="mean")

Categories:
1. Abstract
2. CGS
3. Childrens
4. Family
5. Party
6. Strategy
7. Thematic
8. War


Select categories of interest by entering the corresponding number(s):  4


You selected
Family


Would you like to select a subcategory? (Y/N)  n


Game Choices:
1. Elfenland
2. Bohnanza
3. Catan
4. Basari
5. Wacky Wacky West
6. Can't Stop
7. David & Goliath
8. Perudo
9. Mamma Mia!
10. Lost Cities
11. Fossil
12. Big City
13. Show Manager
14. Star Wars: Episode 1 – Clash of the Lightsabers
15. Igel Ärgern
16. Detroit-Cleveland Grand Prix
17. Klunker
18. Colossal Arena
19. Mystery Rummy: Jack the Ripper
20. Wettstreit der Baumeister
21. Auf Achse
22. I'm the Boss!
23. Guillotine
24. Hoity Toity
25. Money!
26. Take it Easy!
27. Cape Horn
28. Pit
29. Andromeda
30. Metropolis
31. Buy Low Sell High
32. PitchCar
33. Take 5!
34. Was sticht?
35. Wildlife Adventure
36. Before I Kill You, Mister Spy...
37. En Garde
38. Family Business
39. For Sale
40. What the Heck?
41. Give Me the Brain!
42. Risk
43. Knights of Charlemagne
44. Devil Bunny Needs a Ham
45. Die Siedler von Nürnberg
46. Dolce Vita
47. Nicht die Bohne!
48. Fill or Bust
49. Buried Treasure
50. Manhattan
51. Entdecker
52. Café International
53. A la carte
54. High-Bid
55. High Soc

Enter the Game Numbers of your favorite Games separated by spaces:  1 2 3



Your Favorite Game(s):
Elfenland
Bohnanza
Catan


Enter the Game Numbers of your least favorite Games separated by spaces:  4



Your Least Favorite Game(s):
Basari


Would you like recommendations for only the categories you have chosen? (Y/N)  y
